In [1]:
import pandas_ta as ta
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import time
import math
import http.client
import json
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import http.client
import json
import pandas_ta as ta
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import time
import math
from talipp.indicators import EMA, SMA, Stoch, RSI
from talipp.ohlcv import OHLCVFactory

In [2]:
def stop_loss_value(price, position , stop_loss, balance ,direction, leverage):
    '''
    stop_loss_value(price, position , stop_loss, balance ,"BUY", leverage)
    position + percentage of instrument being bought
    price = price of an instrument like bitcoin
    balance = account balance
    stop_loss = perentage like  0.01
    direction "BUY", "SELL"
    '''
    temp= 0
    stop_loss_value = stop_loss*balance
    position_value = position * price
    if direction == "BUY":
            temp =position_value - stop_loss_value
    elif direction == "SELL":
            temp = position_value +stop_loss_value
    stop_loss_price = (temp/position_value)*price
    return(stop_loss_price)

In [3]:
def convert(seconds):
    minutes = seconds // 60
    return minutes

In [4]:
def ping_server_till_next_hour(keys):
    # Calculate the minutes until the next hour using the get_hour_pause function
    minutes_till_next_hour = convert(get_hour_pause())

    # Calculate the number of iterations based on 8-minute intervals
    iterations = minutes_till_next_hour // 8

    # Loop through the iterations and ping the server
    for i in range(iterations):
        # Ping the server using the provided keys
        ping(keys)

        # Sleep for 480 seconds (8 minutes) before the next iteration
        time.sleep(480)


In [2]:
def ping_server_till_next_30min(keys):
    # Calculate the minutes until the next 30 minutes using the get_30min_pause function
    minutes_till_next_30min = convert(get_30min_pause())

    # Calculate the number of iterations based on 8-minute intervals
    iterations = minutes_till_next_30min // 8

    # Loop through the iterations and ping the server
    for i in range(iterations):
        # Ping the server using the provided keys
        ping(keys)

        # Sleep for 480 seconds (8 minutes) before the next iteration
        time.sleep(480)

In [5]:
def get_daily_trading_pause():
    # Calculate the time difference of one hour
    delta = timedelta(hours=1)

    # Get the current date and time
    now = datetime.now()

    # Calculate the time of the next trading day, setting seconds and microsecond to zero,
    # minute to 35, and hour to 15 (3:35 PM)
    next_hour = (now + delta).replace(microsecond=0, second=0, minute=35, hour=15)

    # Calculate the time difference (pause) until the next trading day
    pause = (next_hour - now).seconds

    # Return the calculated pause duration
    return pause

In [6]:
def get_daily_pause():
    # Calculate the time difference of one hour
    delta = timedelta(hours=1)

    # Get the current date and time
    now = datetime.now()

    # Calculate the time of the next day, setting seconds and microsecond to zero,
    # minute to 5, and hour to 0 (midnight)
    next_hour = (now + delta).replace(microsecond=0, second=0, minute=5, hour=0)

    # Calculate the time difference (pause) until the next day
    pause = (next_hour - now).seconds

    # Return the calculated pause duration
    return pause

In [7]:
def get_pause():
    # Get the current date and time
    now = datetime.now()

    # Calculate the time of the next minute, setting seconds and microsecond to zero
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)

    # Calculate the time difference (pause) until the next minute and round up using math.ceil
    pause = math.ceil((next_min - now).seconds)

    # Print the calculated pause duration for informational purposes
    print(f"Sleep for {pause} seconds")

    # Return the calculated pause duration
    return pause

In [8]:
def get_hour_pause():
    # Calculate the time difference of one hour
    delta = timedelta(hours=1)

    # Get the current date and time
    now = datetime.now()

    # Calculate the time of the next hour, setting seconds and microsecond to zero and minute to 1
    next_hour = (now + delta).replace(microsecond=0, second=0, minute=1)

    # Calculate the time difference (pause) until the next hour
    pause = (next_hour - now).seconds

    # Return the calculated pause duration
    return pause

In [1]:
def get_30min_pause():
    # Get the current time
    current_time = datetime.now()
    # Calculate the time until the next half-hour
    pause = (31 - current_time.minute % 30) * 60 - current_time.second
    return pause

In [9]:
def RSI_Calc(df):
   # Calculate diff
    diff=df.Close.diff().dropna()
    # Ups
    plus=diff.map(lambda x: x if x>0 else 0).rename('Plus')
    # Downs
    minus=diff.map(lambda x: -1*x if x<0 else 0).rename('Minus')
    rsi14=pd.concat([diff,plus,minus],axis=1)
    # init average up
    rsi14['AvgUP'] = rsi14.Plus[:14].sum()/14
    # init average down
    rsi14['AvgDOWN'] = rsi14.Minus[:14].sum()/14
    # calculate AvgUP and AvgDown with the recurrent formula
    for i in range(14,rsi14.Close.size):
        rsi14['AvgUP'].iloc[i]=(rsi14['Plus'].iloc[i] + 13*rsi14['AvgUP'].iloc[i-1])/14
        rsi14['AvgDOWN'].iloc[i]=(rsi14['Minus'].iloc[i] + 13*rsi14['AvgDOWN'].iloc[i-1])/14
    rsi14['RSI']=100 - 100 / (1+rsi14['AvgUP']/rsi14['AvgDOWN'])
    return  rsi14['RSI']

In [ ]:
def generate_Breakout_data(spx_data):
    # Resetting index to use the default integer index
    spx_data.reset_index(inplace=True)

    # Extracting relevant columns and calculating additional features
    prices = spx_data
    prices['O-to-C'] = prices['Close'] - prices['Open']
    prices['OC-20D-Mean'] = prices['O-to-C'].rolling(20).mean()
    prices['OC-%-from-20D-Mean'] = 100 * (prices['O-to-C'] - prices['OC-20D-Mean']) / prices['OC-20D-Mean']
    prices['MaxOC_Prev10'] = prices['O-to-C'].rolling(10).max()
    prices['Volume-20D-Mean'] = prices['Volume'].rolling(20).mean()
    prices['Volume-%-from-20D-Mean'] = 100 * (prices['Volume'] - prices['Volume-20D-Mean']) / prices['Volume-20D-Mean']
